# Analyzing the Distances Between Beams

Start off by importing any necessary packages. Check the README for the versions of each of the packages being used.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from tabulate import tabulate
from astropy.time import Time
from tqdm import tqdm
import csv
import math

Open up the .pickle files. The 4_beams.pkl file has all of the hits that are present in 4 of the beams in a given field of view. The same is true for 5_beams.pkl but the hit is present in 5 of the beams. However, there is a caveat here in that all of the hits that were missing an incoherent beam were excluded. These were assumed to be low intensity RFI, which when the average was taken to create the incoherent beam were excluded but were still classified as being hits in 4-5 of the coherent beams.

In [4]:
with open('../../4_beams.pkl', 'rb') as f:
    hit_4 = pickle.load(f)
print(hit_4.shape)
file_path_4 = hit_4["file_path"]
unique_file_path_4 = np.unique(file_path_4) #the array of unique fields of view

with open('../../5_beams.pkl', 'rb') as f:
    hit_5 = pickle.load(f)
print(hit_5.shape)
file_path_5 = hit_5["file_path"]
unique_file_path_5 = np.unique(file_path_5) #the array of unique fields of view

(73840, 24)
(181150, 24)


This data was taken at S band with the VLA in the B configuration. This would mean that the beam size should be 2.1 arcseconds. So here's my plan:

- Clump together each hit 
- Find the distances between each of the coordinates in RA and DEC
- Convert these distances to arcseconds

Now there a few other things to consider here. The center of the incoherent beam is usually offset from where all of the coherent beams are, so the distances should be calculated for each of the points aside from the incoherent beam so that it does not cause issues with the distances. We will then filter out any of them that have a distance greater than 6.3 arcseconds.

In [16]:
right_ascension = hit_5["ra"]
declination = hit_5["dec"]
beam = hit_5["beam"]

print(beam[2], beam[1])
print((right_ascension[1]-right_ascension[2])*math.pi*3600/180)
print((declination[1]-declination[2])*math.pi*3600/180)
print(math.sqrt(((right_ascension[1]-right_ascension[2])*math.pi*3600/180)**2+((declination[1]-declination[2])*math.pi*3600/180)**2))

2.0 4.0
0.5621356454823611
-5.987875597742031
6.014204075183711


So I'm going to make a for loop that goes through all of the clumps, and then does the above calculation for each of the combinations, except for the coherent beam. To do this, once I pull the clump, I'll sort it by the beam and then only look at the combinations that occur within beams 0-4. I will append all of the distances to an array for the specific clump. Then if any of the values in the array are greater than 6.3, the clump will not get appended to the dataframe. 